In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [46]:
def read_files(freq_band):
    scpc = "_scpc_channel_data"
    mcc = "_mcc_packet_lost"
    sat = "_sat_packet_lost"

    data = pd.read_csv(freq_band + scpc + '.txt', sep=",", header=0)
    real = data.head()

    f = open(freq_band + mcc + '.txt', "r")
    print(f.read()) 
    f.close()

    f = open(freq_band + sat + '.txt', "r")
    print(f.read()) 
    f.close()

    print("Average Atmospheric Loss (dB):", np.average(real['atmosphericLoss_dB']))
    print("Stddev Atmospheric Loss (dB):", np.std(real['atmosphericLoss_dB']))
    print()

    # Calculate power_dBm stats for every 5 lines (uplink/downlink pairs)
    fspl_uplink = []
    fspl_downlink = []
    
    for i in range(len(data)):
        if str(data.iloc[i,0])[-2] == '.':
            fspl_uplink.append(data.iloc[i,2])
        else:
            fspl_downlink.append(data.iloc[i,2])

    print(f"Average uplink FSPL (dB): {np.average(fspl_uplink):.3f}")
    print(f"Stddev uplink FSPL (dB): {np.std(fspl_uplink):.3f}")

    print(f"Average downlink FSPL (dB): {np.average(fspl_downlink):.3f}")
    print(f"Stddev downlink FSPL (dB): {np.std(fspl_downlink):.3f}")
    print()

    # Calculate power_dBm stats for every 5 lines (uplink/downlink pairs)
    power_dBm_uplink = []
    power_dBm_downlink = []
    
    for i in range(len(data)):
        if str(data.iloc[i,0])[-2] == '.':
            power_dBm_uplink.append(data.iloc[i,3])
        else:
            power_dBm_downlink.append(data.iloc[i,3])

    print(f"Average uplink power (dBm): {np.average(power_dBm_uplink):.3f}")
    print(f"Stddev uplink power (dBm): {np.std(power_dBm_uplink):.3f}")

    print(f"Average downlink power (dBm): {np.average(power_dBm_downlink):.3f}")
    print(f"Stddev downlink power (dBm): {np.std(power_dBm_downlink):.3f}")

In [47]:
read_files("C-Band-Sunny")

MCC 0: Packets Lost = 0
MCC 1: Packets Lost = 0
MCC 2: Packets Lost = 0
MCC 3: Packets Lost = 0
MCC 4: Packets Lost = 0

Satellite: Packets Lost = 0

Average Atmospheric Loss (dB): 0.0
Stddev Atmospheric Loss (dB): 0.0

Average uplink FSPL (dB): 199.220
Stddev uplink FSPL (dB): 0.091
Average downlink FSPL (dB): 195.739
Stddev downlink FSPL (dB): 0.106

Average uplink power (dBm): -76.667
Stddev uplink power (dBm): 0.091
Average downlink power (dBm): -108.307
Stddev downlink power (dBm): 0.155


In [48]:
read_files("C-Band-Rainy")

MCC 0: Packets Lost = 0
MCC 1: Packets Lost = 0
MCC 2: Packets Lost = 0
MCC 3: Packets Lost = 0
MCC 4: Packets Lost = 0

Satellite: Packets Lost = 0

Average Atmospheric Loss (dB): 0.4953058
Stddev Atmospheric Loss (dB): 0.007975233699397148

Average uplink FSPL (dB): 199.220
Stddev uplink FSPL (dB): 0.091
Average downlink FSPL (dB): 195.741
Stddev downlink FSPL (dB): 0.106

Average uplink power (dBm): -77.163
Stddev uplink power (dBm): 0.097
Average downlink power (dBm): -109.001
Stddev downlink power (dBm): 0.162


In [49]:
read_files("C-Band-HeavyRain")

MCC 0: Packets Lost = 0
MCC 1: Packets Lost = 0
MCC 2: Packets Lost = 0
MCC 3: Packets Lost = 0
MCC 4: Packets Lost = 0

Satellite: Packets Lost = 0

Average Atmospheric Loss (dB): 1.260162
Stddev Atmospheric Loss (dB): 0.020289870773368662

Average uplink FSPL (dB): 199.220
Stddev uplink FSPL (dB): 0.091
Average downlink FSPL (dB): 195.741
Stddev downlink FSPL (dB): 0.107

Average uplink power (dBm): -77.928
Stddev uplink power (dBm): 0.107
Average downlink power (dBm): -110.070
Stddev downlink power (dBm): 0.170


In [50]:
read_files("Ku-Band-Sunny")

MCC 0: Packets Lost = 912
MCC 1: Packets Lost = 1013
MCC 2: Packets Lost = 955
MCC 3: Packets Lost = 992
MCC 4: Packets Lost = 916

Satellite: Packets Lost = 0

Average Atmospheric Loss (dB): 0.0
Stddev Atmospheric Loss (dB): 0.0

Average uplink FSPL (dB): 206.531
Stddev uplink FSPL (dB): 0.068
Average downlink FSPL (dB): 205.198
Stddev downlink FSPL (dB): 0.066

Average uplink power (dBm): -83.978
Stddev uplink power (dBm): 0.068
Average downlink power (dBm): -125.076
Stddev downlink power (dBm): 0.126


In [51]:
read_files("Ku-Band-Rainy")

MCC 0: Packets Lost = 892
MCC 1: Packets Lost = 948
MCC 2: Packets Lost = 985
MCC 3: Packets Lost = 937
MCC 4: Packets Lost = 838

Satellite: Packets Lost = 0

Average Atmospheric Loss (dB): 3.431838
Stddev Atmospheric Loss (dB): 0.023819631735188498

Average uplink FSPL (dB): 206.531
Stddev uplink FSPL (dB): 0.068
Average downlink FSPL (dB): 205.197
Stddev downlink FSPL (dB): 0.065

Average uplink power (dBm): -87.410
Stddev uplink power (dBm): 0.083
Average downlink power (dBm): -130.918
Stddev downlink power (dBm): 0.137


In [52]:
read_files("Ku-Band-HeavyRain")

MCC 0: Packets Lost = 890
MCC 1: Packets Lost = 965
MCC 2: Packets Lost = 928
MCC 3: Packets Lost = 978
MCC 4: Packets Lost = 885

Satellite: Packets Lost = 0

Average Atmospheric Loss (dB): 8.731318
Stddev Atmospheric Loss (dB): 0.06059630199937978

Average uplink FSPL (dB): 206.531
Stddev uplink FSPL (dB): 0.068
Average downlink FSPL (dB): 205.198
Stddev downlink FSPL (dB): 0.066

Average uplink power (dBm): -92.709
Stddev uplink power (dBm): 0.112
Average downlink power (dBm): -139.942
Stddev downlink power (dBm): 0.164


In [53]:
read_files("Ka-Band-Sunny")

MCC 0: Packets Lost = 838
MCC 1: Packets Lost = 894
MCC 2: Packets Lost = 864
MCC 3: Packets Lost = 886
MCC 4: Packets Lost = 881

Satellite: Packets Lost = 0

Average Atmospheric Loss (dB): 0.0
Stddev Atmospheric Loss (dB): 0.0

Average uplink FSPL (dB): 213.131
Stddev uplink FSPL (dB): 0.062
Average downlink FSPL (dB): 209.618
Stddev downlink FSPL (dB): 0.061

Average uplink power (dBm): -90.578
Stddev uplink power (dBm): 0.062
Average downlink power (dBm): -136.096
Stddev downlink power (dBm): 0.121


In [54]:
read_files("Ka-Band-Rainy")

MCC 0: Packets Lost = 0
MCC 1: Packets Lost = 0
MCC 2: Packets Lost = 0
MCC 3: Packets Lost = 0
MCC 4: Packets Lost = 0

Satellite: Packets Lost = 5705

Average Atmospheric Loss (dB): 19.70292
Stddev Atmospheric Loss (dB): 0.06395074354532525

Average uplink FSPL (dB): 213.131
Stddev uplink FSPL (dB): 0.062
Average downlink FSPL (dB): nan
Stddev downlink FSPL (dB): nan

Average uplink power (dBm): -110.281
Stddev uplink power (dBm): 0.103
Average downlink power (dBm): nan
Stddev downlink power (dBm): nan


C:\Users\guido\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
C:\Users\guido\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\guido\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\guido\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divi

In [55]:
read_files("Ka-Band-HeavyRain")

MCC 0: Packets Lost = 0
MCC 1: Packets Lost = 0
MCC 2: Packets Lost = 0
MCC 3: Packets Lost = 0
MCC 4: Packets Lost = 0

Satellite: Packets Lost = 5625

Average Atmospheric Loss (dB): 50.12836
Stddev Atmospheric Loss (dB): 0.1627194591928089

Average uplink FSPL (dB): 213.131
Stddev uplink FSPL (dB): 0.062
Average downlink FSPL (dB): nan
Stddev downlink FSPL (dB): nan

Average uplink power (dBm): -140.706
Stddev uplink power (dBm): 0.192
Average downlink power (dBm): nan
Stddev downlink power (dBm): nan


C:\Users\guido\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
C:\Users\guido\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\guido\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\guido\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divi

In [60]:
read_files("C-Band-Variable")

MCC 0: Packets Sent = 18798
MCC 0: Packets Lost = 7731
MCC 1: Packets Sent = 18797
MCC 1: Packets Lost = 0
MCC 2: Packets Sent = 18797
MCC 2: Packets Lost = 7172
MCC 3: Packets Sent = 18797
MCC 3: Packets Lost = 0
MCC 4: Packets Sent = 18797
MCC 4: Packets Lost = 0

Satellite: Packets Sent = 85632
Satellite: Packets Lost = 8348

Average Atmospheric Loss (dB): 0.27218700000000007
Stddev Atmospheric Loss (dB): 0.13623569333328178

Average uplink FSPL (dB): 199.220
Stddev uplink FSPL (dB): 0.091
Average downlink FSPL (dB): 195.749
Stddev downlink FSPL (dB): 0.108

Average uplink power (dBm): -80.314
Stddev uplink power (dBm): 5.436
Average downlink power (dBm): -112.038
Stddev downlink power (dBm): 6.229
